In [28]:
from yahoo_fin.stock_info import *

In [65]:
def importlibs():
    import time
    from datetime import timedelta
    import datetime as dt
    import yahoo_fin.stock_info as si
    import pandas as pd
    import numpy as np

def prev_weekday(adate):
    importlibs()
    #https://stackoverflow.com/questions/12053633/previous-weekday-in-python
    adate -= timedelta(days=1)
    while adate.weekday() > 4: # Mon-Fri are 0-4
        adate -= timedelta(days=1)
    return adate

def check_week():
    importlibs()
    
    todaysDate = dt.date.today()
    date_today=todaysDate.weekday()

    if date_today in range(0,4):
        todaysDate = dt.date.today()
    else:
        todaysDate=prev_weekday(todaysDate)

    yesterday = todaysDate - timedelta(days = 1)
    yesterday = yesterday.weekday()

    if yesterday in range(1,5):
        yesterday = todaysDate - timedelta(days = 1)
    else:
        yesterday=prev_weekday(todaysDate)
    
    response={"today":todaysDate,"yesterday":yesterday}
    
    return(response)

class ticker:
    
    def __init__(self, ticker):
        
        importlibs()
        
        # Validamos que el dia de ejecucion sea valido
        self.todaysDate=check_week()["today"]
        self.yesterday=check_week()["yesterday"]
        self.today=self.todaysDate.strftime('%m/%d/%Y')
        
        self.name=ticker
        print("Creating ticker ...")
    
    def getLatest(self):
        
        # Obtenemos los ultimos registros para el dia actual
        data=get_data(self.name , start_date = self.today)
        print(str(self.name) + str(" was created, date:") + str(self.today) )
        
        # Obtenemos ultimo precio dsiponible para el instrumento
        data["last_price"]=si.get_live_price(self.name)
        
        # Seleccionamos los campos y reiniciamos el indice
        data=data[["open","high","low","last_price"]].reset_index()        
        data = data.rename(columns={'index':'date'})
        
        # Hacemos calculo de variacion respecto a la apertura
        data["change"]=((data["last_price"]/data["open"])-1)*100
        data["price_change"]=data["last_price"]-data["open"]
        
        return(data)
    
    def getDays(self,i):
        
        self.daysAgo = self.todaysDate - timedelta(days = i)
        data = get_data(self.name , start_date = self.daysAgo, end_date= self.today)
        data = data.reset_index()
        data = data.rename(columns={'index':'date'}) 
        
        data_day = get_data(self.name , start_date = self.today)
        
        date = dt.datetime.strptime(self.today, "%m/%d/%Y")
        
        data_day["date"] = date
        
        data=pd.concat([data, data_day],ignore_index=True)
        
        return(data)
    
    def getWeeks(self,i):
        
        self.daysAgo = self.todaysDate - timedelta(weeks = i)
        data = get_data(self.name , start_date = self.daysAgo, end_date= self.today)
        data = data.reset_index()
        data = data.rename(columns={'index':'date'})
        
        data_day = get_data(self.name , start_date = self.today)
        
        date = dt.datetime.strptime(self.today, "%m/%d/%Y")
        
        data_day["date"] = date
        
        data = pd.concat([data, data_day],ignore_index=True)
        
        return(data)

In [66]:
ticker("QQQ").getWeeks(1)

Creating ticker ...


,date,open,high,low,close,adjclose,volume,ticker
0,2020-08-05,270.880005,271.519989,269.959991,271.049988,271.049988,21685900,QQQ
1,2020-08-06,270.850006,274.980011,270.190002,274.640015,274.640015,28048200,QQQ
2,2020-08-07,274.079987,274.880005,269.250000,271.470001,271.470001,44944900,QQQ
3,2020-08-10,271.769989,271.980011,266.670013,270.309998,270.309998,35088400,QQQ
4,2020-08-11,269.109985,270.440002,264.630005,265.190002,265.190002,42930800,QQQ
5,2020-08-12,267.369995,272.839996,267.369995,271.859985,271.859985,36606291,QQQ


In [ ]:
ticker("QQQ").getLatest()["last_price"][0]